In [151]:
import pandas as pd
import tensorflow
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import cv2 as cv


In [152]:
iterator_row_index = 0;
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)

In [153]:
def open_image(row):
    global iterator_row_index;
    print("row: ", iterator_row_index);
    iterator_row_index += 1;
    image = np.zeros((784));

    it = 0;
    col = "pixel";
    for it in range(784):
        image[it] = row[col + str(it)];  
        it += 1;

    image = image.reshape((-1, 28));

    gen = np.array(image ,dtype=np.uint8)

    edges = cv.Canny(gen,100,200)
    edges = edges.reshape((-1));
    
    it = 0;
    for it in range(784):
        if(edges[it] == 255): row[col + str(it)] = 1;
        else: row[col + str(it)] = 0;
        it += 1;
 
    return row; 



In [154]:
figure = plt.figure(figsize=(8, 8))


<Figure size 800x800 with 0 Axes>

In [155]:
df = pd.read_csv("Edge_and_normalization_save.csv");
display(df);

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10236,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10237,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10238,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
real_labels = pd.DataFrame();
real_labels['0'] = df['label'].apply(lambda x: 1 if x == 0 else 0);
real_labels['1'] = df['label'].apply(lambda x: 1 if x == 1 else 0);
real_labels['2'] = df['label'].apply(lambda x: 1 if x == 2 else 0);
real_labels['3'] = df['label'].apply(lambda x: 1 if x == 3 else 0);
real_labels['4'] = df['label'].apply(lambda x: 1 if x == 4 else 0);
real_labels['5'] = df['label'].apply(lambda x: 1 if x == 5 else 0);
real_labels['6'] = df['label'].apply(lambda x: 1 if x == 6 else 0);
real_labels['7'] = df['label'].apply(lambda x: 1 if x == 7 else 0);
real_labels['8'] = df['label'].apply(lambda x: 1 if x == 8 else 0);
real_labels['9'] = df['label'].apply(lambda x: 1 if x == 9 else 0);


In [157]:
display(real_labels);

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
10235,0,0,0,0,0,1,0,0,0,0
10236,0,0,0,0,0,0,1,0,0,0
10237,0,0,0,0,0,0,0,1,0,0
10238,0,0,0,0,0,0,0,0,1,0


In [158]:

device = torch.device("mps")
print(device)

# load the dataset, split into input (X) and output (y) variables
dataset = df;



mps


In [159]:
X = df.drop(columns=['label']).to_numpy();
y = real_labels.to_numpy();

X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

print(X_tensor);
print(y_tensor);

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0')
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='mps:0')


In [160]:
model = nn.Sequential(
    nn.Linear(784, 784),
    nn.ReLU(),
    nn.Linear(784, 784),
    nn.ReLU(),
    nn.Linear(784, 1000),
    nn.ReLU(),
    nn.Linear(1000, 784),
    nn.ReLU(),
    nn.Linear(784, 10),
    nn.Softmax()
)
model.to(device);
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=784, bias=True)
  (1): ReLU()
  (2): Linear(in_features=784, out_features=784, bias=True)
  (3): ReLU()
  (4): Linear(in_features=784, out_features=1000, bias=True)
  (5): ReLU()
  (6): Linear(in_features=1000, out_features=784, bias=True)
  (7): ReLU()
  (8): Linear(in_features=784, out_features=10, bias=True)
  (9): Softmax(dim=None)
)


In [161]:
loss_fn   = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [162]:
n_epochs = 20
batch_size = 10

In [163]:
for epoch in range(n_epochs):
    for i in range(0, len(X_tensor), batch_size):
        if(i % 4000 == 0): 
            print(i / len(X_tensor) * 100, '% epoch', epoch);
        
        Xbatch = X_tensor[i:i+batch_size].to(device)
        ybatch = y_tensor[i:i+batch_size].to(device);
        
        y_pred = model(Xbatch).to(device);

        loss = loss_fn(y_pred, ybatch).to(device);
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Finished epoch {epoch}, latest loss {loss}')

0.0 % epoch 0


/Users/tudor/Documents/Work AI/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


39.0625 % epoch 0
78.125 % epoch 0
Finished epoch 0, latest loss 0.014514799229800701
0.0 % epoch 1
39.0625 % epoch 1
78.125 % epoch 1
Finished epoch 1, latest loss 0.016701756045222282
0.0 % epoch 2
39.0625 % epoch 2
78.125 % epoch 2
Finished epoch 2, latest loss 0.0013718712143599987
0.0 % epoch 3
39.0625 % epoch 3
78.125 % epoch 3
Finished epoch 3, latest loss 7.028794698271668e-06
0.0 % epoch 4
39.0625 % epoch 4
78.125 % epoch 4
Finished epoch 4, latest loss 0.00030132822575978935
0.0 % epoch 5
39.0625 % epoch 5
78.125 % epoch 5
Finished epoch 5, latest loss 0.0002367824490647763
0.0 % epoch 6
39.0625 % epoch 6
78.125 % epoch 6
Finished epoch 6, latest loss 0.0004395816067699343
0.0 % epoch 7
39.0625 % epoch 7
78.125 % epoch 7
Finished epoch 7, latest loss 1.1364032616256736e-05
0.0 % epoch 8
39.0625 % epoch 8
78.125 % epoch 8
Finished epoch 8, latest loss 7.640843335821046e-08
0.0 % epoch 9
39.0625 % epoch 9
78.125 % epoch 9
Finished epoch 9, latest loss 0.005746121052652597
0.0 %

In [164]:
with torch.no_grad():
    y_pred = model(X_tensor).to(device);
    y_pred.to(device);

print(y_pred.shape);
print(y_tensor.shape);
accuracy = (y_pred.round() == y_tensor).float().mean()
print(f"Accuracy {accuracy}")

torch.Size([10240, 10])
torch.Size([10240, 10])
Accuracy 0.9829882383346558


In [172]:
test = pd.read_csv("test.csv");
test.drop(columns='id', inplace=True);
test = test.apply(open_image, axis=1);
arr = test.to_numpy();
print(arr);


row:  8458
row:  8459
row:  8460
row:  8461
row:  8462
row:  8463
row:  8464
row:  8465
row:  8466
row:  8467
row:  8468
row:  8469
row:  8470
row:  8471
row:  8472
row:  8473
row:  8474
row:  8475
row:  8476
row:  8477
row:  8478
row:  8479
row:  8480
row:  8481
row:  8482
row:  8483
row:  8484
row:  8485
row:  8486
row:  8487
row:  8488
row:  8489
row:  8490
row:  8491
row:  8492
row:  8493
row:  8494
row:  8495
row:  8496
row:  8497
row:  8498
row:  8499
row:  8500
row:  8501
row:  8502
row:  8503
row:  8504
row:  8505
row:  8506
row:  8507
row:  8508
row:  8509
row:  8510
row:  8511
row:  8512
row:  8513
row:  8514
row:  8515
row:  8516
row:  8517
row:  8518
row:  8519
row:  8520
row:  8521
row:  8522
row:  8523
row:  8524
row:  8525
row:  8526
row:  8527
row:  8528
row:  8529
row:  8530
row:  8531
row:  8532
row:  8533
row:  8534
row:  8535
row:  8536
row:  8537
row:  8538
row:  8539
row:  8540
row:  8541
row:  8542
row:  8543
row:  8544
row:  8545
row:  8546
row:  8547
row:  8548

In [173]:
X_test_tensor = torch.tensor(arr, dtype=torch.float32).to(device);

In [174]:
file = open("ansGPU.csv", 'w');

In [175]:
mat = model(X_test_tensor).cpu().data.numpy();
print(mat);


for i in range(len(mat)):
    mx = mat[i].argmax();
    file.write(str(i + 1) + ',' + str(mx) + "\n");


[[1.7392255e-15 1.0269641e-12 9.6829576e-13 ... 1.6407719e-07
  1.2626425e-24 1.0401719e-16]
 [3.5354793e-01 1.2243209e-02 1.1834417e-06 ... 3.5019807e-04
  2.6081877e-06 4.6252907e-08]
 [0.0000000e+00 1.6821317e-29 1.0000000e+00 ... 1.7847729e-32
  0.0000000e+00 0.0000000e+00]
 ...
 [3.2968108e-09 1.3011935e-08 1.8295420e-07 ... 6.7930611e-13
  8.4134755e-11 3.2430297e-10]
 [8.7048402e-06 2.9481098e-17 2.8142579e-15 ... 2.5409850e-01
  6.8330296e-15 3.5108285e-05]
 [3.2139092e-15 6.3278272e-12 2.5372430e-11 ... 2.0594449e-08
  3.2744372e-22 3.9421355e-16]]


/Users/tudor/Documents/Work AI/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
